In [ ]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import root_mean_squared_error

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso
from sklearn.svm import SVR

import matplotlib.pyplot as plt
import seaborn as sns

import optuna

import joblib
import json

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows = 500
pd.options.display.max_rows = 500

## Config

In [ ]:
class Config:
    target_col = 'pm2_5'
    n_splits = 4
    random_state = 42
    id_col = 'id'
    missing_threshold = 0.7
    top_features = 70
    clip_threshold = 0.97
    corr_threshold = 0.9

    # Default hyperparameters
    cat_params = {
        'iterations': 1000,
        'learning_rate': 0.03,
        'depth': 6,
        'eval_metric': 'RMSE',
        'random_seed': random_state,
        'early_stopping_rounds': 250,
        'verbose': 100
    }
    lgb_params = {
        'n_estimators': 100,
        'learning_rate': 0.1,
        'max_depth': -1,
        'random_state': random_state,
        'verbosity': -1
    }
    xgb_params = {
        'n_estimators': 100,
        'learning_rate': 0.3,
        'max_depth': 6,
        'random_state': random_state,
        'objective': 'reg:squarederror'
    }
    lasso_params = {'alpha': 0.001, 'random_state': random_state}
    svr_params = {'C': 1.0, 'epsilon': 0.1}

## Load data

In [ ]:
# Load data
train = pd.read_csv('dataset/Train.csv')
test = pd.read_csv('dataset/Test.csv')

In [ ]:
# Remove columns with too many missing values
train = train.loc[:, train.isnull().mean() < Config.missing_threshold]
test = test.loc[:, test.isnull().mean() < Config.missing_threshold]  

In [ ]:
# Create GroupKFold
def create_folds(data):
    data['folds'] = -1
    gkf = GroupKFold(n_splits=Config.n_splits)
    for fold, (train_idx, val_idx) in enumerate(gkf.split(X=data, groups=data['city']), start=1):
        data.loc[val_idx, 'folds'] = fold
    return data

train = create_folds(train)

### Explore data

In [ ]:
train.head()

In [ ]:
# Calculate PM2.5 standard deviation per location
location_variance = train.groupby('city')[Config.target_col].std().reset_index()
location_variance.columns = ['city', 'pm2_5_std']
location_variance = location_variance.sort_values(by='pm2_5_std', ascending=False).reset_index(drop=True)
location_variance

In [ ]:
# Check for outliers in the target variable
plt.figure(figsize = (22, 10))
sns.boxplot(train.pm2_5)
plt.title('Boxplot showing outliers - target variable')
plt.show()

In [ ]:
# Plot correlations between numerical features and target
train_num_df = train.select_dtypes(include=['number'])
top10_corrs = abs(train_num_df.corr()['pm2_5']).sort_values(ascending=False).head(10)
corr = train_num_df[list(top10_corrs.index)].corr()

plt.figure(figsize=(10, 8), dpi=150)
sns.heatmap(
    corr, cmap='RdYlGn', annot=True, center=0, fmt=".2f", 
    annot_kws={"size": 9}
)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title('Correlations between the target and other numeric variables', pad=15, fontdict={'size': 14})
plt.tight_layout()
plt.show()

## Feature engineering

In [ ]:
# Feature engineering
def feature_engineering(train, test):
    le = LabelEncoder()
    data = pd.concat([train, test])
    data['location'] = data['site_latitude'].astype('str') + '_' + data['site_longitude'].astype('str')
    data = data.sort_values(by = ['city','location', 'date', 'hour'])
    categorical_cols = data.select_dtypes(include='object').columns.tolist()
    categorical_cols = [col for col in categorical_cols if col not in ['date', 'id', 'city', 'country']]
    print(f'Categorical columns: {categorical_cols}')

    # Date features
    data['date'] = pd.to_datetime(data['date'])
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.isocalendar().week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    data['is_weekend'] = data['dayofweek'].isin([5,6]).astype(int)

    numerical_cols = data.select_dtypes(exclude='object').columns.tolist()
    numerical_cols.remove(Config.target_col)
    numerical_cols.remove('folds')
    numerical_cols.remove('hour')
    numerical_cols.remove('site_latitude')
    numerical_cols.remove('site_longitude') 
    print(f'Numerical columns: {numerical_cols}')

    # Fill in missing values by forward and backward fill within each city and location
    nan_cols = [col for col in numerical_cols if data[col].isnull().sum() > 0 and col not in [Config.target_col, "folds"]]
    for col in nan_cols:
        data[col] = (
            data.groupby(["city", "location"])[col]
                .transform(lambda x: x.ffill().bfill())
                .fillna(data[col].median())  # global fallback
            )

    # Encode categorical features
    for col in categorical_cols + ['date']:
        data[col] = le.fit_transform(data[col])

    # Split back into train and test
    train  = data[data['id'].isin(train['id'].unique())]
    test = data[data['id'].isin(test['id'].unique())]

    features = [col for col in data.columns if col not in 
                [Config.target_col, Config.id_col, 'folds', 'country', 'city', 'site_id', 'site_latitude', 'site_longitude']]
  
    return train, test, features

train, test, features = feature_engineering(train, test)

In [ ]:
test


In [ ]:
train.isna().sum()


## Feature selection

### Embedded method

In [ ]:
def top_k_feature_selection(X, y, k=Config.top_features):
    """
    Select top-k features based on CatBoost feature importance.
    
    Parameters
    ----------
    X : pd.DataFrame
        Feature matrix
    y : pd.Series or np.ndarray
        Target vector
    k : int
        Number of top features to select
    """
    model = CatBoostRegressor(**Config.cat_params)
    model.fit(X, y)

    feature_importances = model.get_feature_importance(prettified=True)
    top_features = feature_importances.head(k)['Feature Id'].tolist()

    return top_features

In [ ]:
# Drop highly correlated features
def drop_highly_correlated_features(X, threshold=0.9):
    corr_matrix = X.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    print(f'Dropping {len(to_drop)} highly correlated features: {to_drop}')
    reduced_features = [feature for feature in X.columns.tolist() if feature not in to_drop]
    return reduced_features

## Model ensemble

In [ ]:
class EnsembleModel:
    def __init__(self, top_features=30, corr_threshold=0.9, clip_threshold=0.99):
        self.top_features = top_features
        self.corr_threshold = corr_threshold
        self.clip_threshold = clip_threshold
        self.models = None
        self.reduced_features = None
        self.best_params = None

    def _clip_target(self, y):
        clip_val = np.quantile(y, self.clip_threshold)
        return np.where(y >= clip_val, clip_val, y)

    def _feature_selection(self, X, y):
        top_feats = top_k_feature_selection(X, y, k=self.top_features)
        self.reduced_features = drop_highly_correlated_features(X[top_feats], threshold=self.corr_threshold)

    def _fit_models(self, X, y, models):
        for name, model in models.items():
            model.fit(X[self.reduced_features], y)
        return models

    def cross_val_fit(self, X, y, folds, models, X_test=None):
        """Run CV to evaluate hyperparameters; returns OOF, test predictions, mean RMSE."""
        oof_preds = np.zeros(len(y))
        fold_test_preds = [] if X_test is not None else None
        fold_rmse_list = []

        unique_folds = np.unique(folds)
        self._feature_selection(X, y)

        for fold in unique_folds:
            train_idx = folds != fold
            val_idx = folds == fold
            X_train, y_train = X[train_idx], y[train_idx]
            X_val, y_val = X[val_idx], y[val_idx]

            y_train_clipped = self._clip_target(y_train)

            val_preds = np.zeros((len(X_val), len(models)))
            test_preds = np.zeros((len(X_test), len(models))) if X_test is not None else None

            # Train each model
            models_fold = self._fit_models(X_train, y_train_clipped, models)
            for i, (name, model) in enumerate(models_fold.items()):
                val_preds[:, i] = model.predict(X_val[self.reduced_features])
                if X_test is not None:
                    test_preds[:, i] = model.predict(X_test[self.reduced_features])

            # Ensemble
            oof_preds[val_idx] = val_preds.mean(axis=1)
            if X_test is not None:
                fold_test_preds.append(test_preds.mean(axis=1))

            # Fold RMSE
            fold_rmse = np.sqrt(np.mean((y_val - oof_preds[val_idx])**2))
            fold_rmse_list.append(fold_rmse)

        mean_rmse = np.mean(fold_rmse_list)
        final_test_preds = np.mean(fold_test_preds, axis=0) if X_test is not None else None
        return oof_preds, final_test_preds, mean_rmse

    def fit_final(self, X, y, models):
        """Train final ensemble on full training data after hyperparameter selection."""
        y_clipped = self._clip_target(y)
        self._feature_selection(X, y_clipped)
        self.models = self._fit_models(X, y_clipped, models)

    def predict(self, X):
        """Predict using the ensemble (mean of all models)."""
        test_preds = np.zeros((len(X), len(self.models)))
        for i, (name, model) in enumerate(self.models.items()):
            test_preds[:, i] = model.predict(X[self.reduced_features])
        return np.mean(test_preds, axis=1)

ensemble = EnsembleModel(top_features=Config.top_features,
                         corr_threshold=Config.corr_threshold,
                         clip_threshold=Config.clip_threshold)

## Cross-validation with GroupKFold for hyperparameter search

In [ ]:
def objective(trial):
    models = {
        "cat": CatBoostRegressor(
            iterations=1000,
            learning_rate=trial.suggest_float('cat_lr', 0.01, 0.1, log=True),
            depth=trial.suggest_int('cat_depth', 4, 10),
            random_seed=Config.random_state,
            verbose=0,
            early_stopping_rounds=250
        ),
        "lgb": LGBMRegressor(
            n_estimators=100,
            learning_rate=trial.suggest_float('lgb_lr', 0.01, 0.1, log=True),
            max_depth=trial.suggest_int('lgb_depth', 3, 12),
            random_state=Config.random_state,
            verbosity=-1
        ),
        "xgb": XGBRegressor(
            n_estimators=100,
            learning_rate=trial.suggest_float('xgb_lr', 0.01, 0.1, log=True),
            max_depth=trial.suggest_int('xgb_depth', 3, 12),
            random_state=Config.random_state,
            objective='reg:squarederror'
        ),
        "lasso": Lasso(alpha=trial.suggest_float('lasso_alpha', 1e-4, 1.0, log=True),
                       random_state=Config.random_state),
        "svr": SVR(
            C=trial.suggest_float('svr_C', 0.1, 10.0, log=True),
            epsilon=trial.suggest_float('svr_eps', 0.01, 1.0, log=True)
        )
    }

    _, _, mean_rmse = ensemble.cross_val_fit(
        train[features],
        train[Config.target_col].values,
        folds=train['folds'].values,
        models=models
    )
        
    return mean_rmse

# Optimize hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

## Final train on full data and predict on test set

In [ ]:
# Extract best hyperparameters
best_params = study.best_params
Config.cat_params['learning_rate'] = best_params['cat_lr']
Config.cat_params['depth'] = best_params['cat_depth']
Config.lgb_params['learning_rate'] = best_params['lgb_lr']
Config.lgb_params['max_depth'] = best_params['lgb_depth']
Config.xgb_params['learning_rate'] = best_params['xgb_lr']   
Config.xgb_params['max_depth'] = best_params['xgb_depth']
Config.lasso_params['alpha'] = best_params['lasso_alpha']
Config.svr_params['C'] = best_params['svr_C']
Config.svr_params['epsilon'] = best_params['svr_eps']

# Define final models with best hyperparameters
best_models = {
    "cat": CatBoostRegressor(**Config.cat_params),
    "lgb": LGBMRegressor(**Config.lgb_params),
    "xgb": XGBRegressor(**Config.xgb_params),
    "lasso": Lasso(**Config.lasso_params),
    "svr": SVR(**Config.svr_params)
}

ensemble.fit_final(train[features], train[Config.target_col].values, best_models)
final_test_preds = ensemble.predict(test[features])

In [ ]:
# Save best models
folder_name = f"./weights/{study.best_value:.2f}"  # keep 4 decimal places
os.makedirs(folder_name, exist_ok=True)

# Save models
for name, model in best_models.items():
    joblib.dump(model, f"{folder_name}/best_model_{name}.pkl")

# Save reduced feature list
with open(f"{folder_name}/reduced_features.json", "w") as f:
    json.dump(ensemble.reduced_features, f)

In [ ]:
# Plot feature importance from final CatBoost model
plt.figure(figsize=(20, 16))
feature_importances_df = pd.DataFrame(best_models['cat'].feature_importances_, columns=['Importances'])
feature_importances_df['Feature'] = ensemble.reduced_features
sns.barplot(x='Importances', y='Feature', data=feature_importances_df.sort_values(by=['Importances'], ascending=False).head(20))
plt.title('Top 20 Feature Importances')
plt.show()

## Prepare test submission

In [ ]:
def prepare_submission(test, best_test_pred, save_path='output/submission.csv'):
    """Prepare submission file."""
    test[Config.target_col] = best_test_pred
    submission = test[[Config.id_col, Config.target_col]]
    submission.to_csv(save_path, index=False)
    submission.head()
    
prepare_submission(test, final_test_preds)

### Load best models and save submission

In [ ]:
def load_and_predict(test, rmse=27.84, model_names=None):
    """
    Load best models, make predictions on test set, and ensemble them.

    Args:
        test (pd.DataFrame): Test dataset.
        rmse (float): RMSE value to locate model folder.
        model_names (list): List of model names to load. Defaults to ['cat', 'lgb', 'xgb', 'lasso', 'svr'].

    Returns:
        np.ndarray: Ensemble predictions for the test set.
    """
    if model_names is None:
        model_names = ['cat', 'lgb', 'xgb', 'lasso', 'svr']

    # Load models
    folder_name = f"./weights/{rmse:.2f}"
    models = {name: joblib.load(f"{folder_name}/best_model_{name}.pkl") for name in model_names}

    # Load reduced features
    with open(f"{folder_name}/reduced_features.json", "r") as f:
        features = json.load(f)

    # Prepare test predictions
    test_preds = np.zeros((len(test), len(models)))

    # Predict
    for i, (name, model) in enumerate(models.items()):
        test_preds[:, i] = model.predict(test[features])

    # Ensemble with equal weights
    final_test_pred = np.mean(test_preds, axis=1)

    return final_test_pred

final_test_preds = load_and_predict(test, rmse=27.78)

# Save final submission
# prepare_submission(test, final_test_preds)

In [ ]:
# Post processing by multiply with a scale factor
scale_factor = 1  # Adjust this factor based on validation results
final_test_preds_scaled = final_test_preds * scale_factor
prepare_submission(test, final_test_preds_scaled)